# Example of Weak Label Learning with warm up

In [39]:
from datasets import openml_datasets,torch_datasets
from utils import weakener
from models import general_model

In [29]:
DS = openml_datasets.OpenML_Dataset('iris')

In [30]:
print(DS.num_features)
print(DS.num_classes)

4
tensor(3, dtype=torch.int32)


In [31]:
Weak = weakener.Weakener(DS.num_classes)

In [32]:
print(dir(Weak))

['M', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'c', 'generate_M', 'generate_weak', 'generate_wl_priors', 'label_matrix', 'pll_weights', 'virtual_labels', 'virtual_matrix', 'w', 'z']


In [33]:
print(DS.__dict__)

{'dataset': 'iris', 'tr_size': 0.7, 'weak_labels': None, 'batch_size': 64, 'shuffle': True, 'splitting_seed': 47, 'num_classes': tensor(3, dtype=torch.int32), 'train_dataset': <torch.utils.data.dataset.TensorDataset object at 0x0000027806739EB0>, 'test_dataset': <torch.utils.data.dataset.TensorDataset object at 0x0000027806739130>, 'train_num_samples': 105, 'test_num_samples': 45, 'num_features': 4}


In [34]:
Weak.generate_M(model_class='pll')

In [36]:
print(Weak.__dict__)

{'c': tensor(3, dtype=torch.int32), 'M': array([[0.   , 0.375, 0.375],
       [0.375, 0.   , 0.375],
       [0.375, 0.375, 0.   ],
       [0.25 , 0.25 , 0.25 ]]), 'z': None, 'w': None, 'pll_p': 0.5, 'Z': array([[0, 1, 1],
       [1, 0, 1],
       [1, 1, 0],
       [1, 1, 1]]), 'labels': {0: '011', 1: '101', 2: '110', 3: '111'}}


In [40]:
mlp = general_model.MLP(DS.num_features, [DS.num_features, DS.num_features, DS.num_features], DS.num_classes, dropout_p = 0.0, bn = False, seed = 1,
                        layer_init = lambda x: nn.init.kaiming_uniform_(x, a=math.sqrt(5)))

NameError: name 'torch' is not defined